## helpers.py

A file containing classes for Streets, Cars & Intersections

In [ ]:
class Street():
    def __init__(self, street_name, B, E, street_length):
        self.street_name = street_name
        self.B = B # starting intersection ID
        self.E = E # ending intersection ID
        # time it takes for a car to cross this road, reaching Intersection E from B
        self.street_length = street_length 
        # variable we added to see the total number of cars that will be crossing this road.
        self.total_cars = 0
        # The following metric is based on an idea to categorize streets on avenues and small streets
        # so that we can schedule their lights accordingly
        self.avenue_metric = 0
        self.is_avenue = False
    
    def __str__(self):
        return f"Starting from intersection {self.B} ending {self.E} & length {self.street_length} total cars {self.total_cars}"   

    def add_car(self):
        self.total_cars += 1

    def update_avenue_metric(self):
        # the more cars inserting the road and the smallest the street length
        # means that more cars will be waiting on the traffic lights
        self.avenue_metric = self.total_cars / self.street_length


class Car():
    def __init__(self, path):
        self.path = path # number of streets
        self.dc = {}
        for street in path:
            self.dc[street] = 0

    def __str__(self):
        return ', '.join(self.path)

class Intersection():
    def __init__(self, i):
        self.i = i
        self.cars_per_incoming_street = {}
        self.cars_per_outgoing_street = {}
        self.traffic_lights = {}
        self.total_cars = 0

    def add_cars_per_incoming_street(self,street_name, total_cars):
        self.cars_per_incoming_street[street_name] = total_cars

    def add_cars_per_outgoing_street(self,street_name, total_cars):
        self.cars_per_outgoing_street[street_name] = total_cars
        
    # this function sets all traffic lights to one if there is at least one car in this road.
    # also it sorts the traffic lights based on the number of cars that use them. This matters because 
    # the final order determines which street will start becoming green earlier.
    def assign_all_one(self):

        self.total_cars = sum(self.cars_per_incoming_street.values())
        if self.total_cars == 0:
            return

        self.cars_per_incoming_street = {k: v for k, v in sorted(self.cars_per_incoming_street.items(), key=lambda item: item[1], reverse=True)}

        for street in self.cars_per_incoming_street.keys():
            if self.cars_per_incoming_street[street] > 0:
                self.traffic_lights[street] = 1
                
                
    def assign_time_percentage(self):
        self.total_cars = sum(self.cars_per_incoming_street.values())
        if self.total_cars == 0:
            return
        
        percentages = {}
        for street in self.cars_per_incoming_street.keys():
            if self.cars_per_incoming_street[street] > 0:
                percentages[street] = self.cars_per_incoming_street[street] / self.total_cars
        
        # find the min percentage that is not equal to 0
        # assuming we have percentages 0.2, 0.3, 0.5 then factor will be 1/0.2 = 5
        factor = 1 / min(percentages.values())
        
        # sort percentages
        percentages = {k: v for k, v in sorted(percentages.items(), key=lambda item: item[1],reverse = True)}
        
        # so we set traffic lights equal to 1.0, 1.5, 2.5 which become 1, 1, 2 since we are converting them to ints
        for street in percentages.keys():
            p = percentages[street]
            self.traffic_lights[street] = max(1,int(p*factor))


## parse.py

In [ ]:
def parse_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

        D, I, S, V, F = [int(w) for w in lines[0].strip().split()]
        
        streets = {}
        cars = []
        
        for i in range(1, S+1):
            B, E, street_name, street_length = [w for w in lines[i].strip().split()]
            B = int(B)
            E = int(E)
            street_length = int(street_length)
            
            streets[street_name] = Street(street_name, B, E, street_length)
            
        for i in range(S+1, len(lines)): 
             cars.append(Car([w for w in lines[i].strip().split()[1:]]))
            
            
    return D, I, S, V, F, streets, cars

In [ ]:
def submit(filename, intersections):
    with open(filename, 'w') as f:
        
        total_intersections = len(intersections)
        
        f.write(str(total_intersections) + "\n")

        for i in intersections.keys():
            inter = intersections[i]
            f.write(f"{inter.i}\n")
            f.write(f"{len(inter.traffic_lights)}\n")
            for street in inter.traffic_lights.keys():
                f.write(f"{street} {inter.traffic_lights[street]}\n")


In [ ]:
# this function removes the cars whose total duration path exceeds the total time of the simulation
# we compute the path's total duration by parsing streets one by one for every car and then going to the
# dictionary streets to get the street_length
# This operation should not be so much expensive.
def excluding_cars():
    i = 0
    while i < len(cars):
        c = cars[i]
        total_duration = 0

        for street_name in c.path[:]:

            total_duration += streets[street_name].street_length

        if total_duration > D:
            print(f"Car {i} discarded")
            cars.pop(i)
        i = i + 1
                                
# for every road in the path of a car we increase total_cars 
def compute_cars_per_street():
    for c in cars:
        for street_name in c.path[:]:
            streets[street_name].total_cars += 1
                
filenames = ['a', 'b', 'c', 'd', 'e', 'f']
for file in filenames:
    input_filename = "./input/" + file + ".txt"
    output_filename = "./output/" + file + "_out.txt"

    D, I, S, V, F, streets, cars = parse_file(input_filename)
    print("-"*100)
    print(f"Filename: {file}")
    print(f"Duration: {D}, \nTotal Intersections: {I}, \nTotal Streets: {S}")
    print(f"Total Vehicles: {V}, \nScore for every car: {F}")

    excluding_cars()
    compute_cars_per_street()
    
    intersections = {}
    for name in streets.keys():
        i = streets[name].B
        if i not in intersections:
            intersections[i] = Intersection(i)
            
        intersections[i].add_cars_per_outgoing_street(name, streets[name].total_cars)
            
        i = streets[name].E
        if i not in intersections:
            intersections[i] = Intersection(i)
        intersections[i].add_cars_per_incoming_street(name, streets[name].total_cars)
      
    final_intersections = {}
    for i in intersections.keys():
        
#         intersections[i].assign_all_one()
        intersections[i].assign_time_percentage()
        
        if intersections[i].total_cars > 0:
            final_intersections[i] = intersections[i]
        

    submit(output_filename, final_intersections)
